# Converting EEG data into BIDS Format
## Author: Will Decker

Load necessary libraries

In [ ]:
import os
import os.path as op
import shutil
import mne
from mne.datasets import eegbci
from mne_bids import write_raw_bids, BIDSPath, print_dir_tree
from mne_bids.stats import count_events

Load data

In [ ]:
datadir = " "; # path to raw data

print_dir_tree(datadir) # expand/display file tree

# create an empty list to store the raw data
raw_data = []

# loop through the files in the folder
for filename in os.listdir(datadir):
    # check if the file is a BrainVision (.vhdr) file
    if filename.endswith(".vhdr"):
        # construct the full file path
        file_path = os.path.join(datadir, filename)
        # load the data from the BrainVision file
        raw = mne.io.read_raw_brainvision(file_path)
        # add the raw data to the list
        raw_data.append(raw)

# combine the raw data into a single raw object
raw = mne.concatenate_raws(raw_data)

print(raw.info)

Montage

In [ ]:
get_montage = mne.channels.get_builtin_montages(descriptions = True) # get list of standard montages
for montage_name, montage_description in get_montage: 
    print(f'{montage_name}: {montage_description}')
    montage = montage_name
    break

standard_montage = mne.channels.make_standard_montage(montage)
raw.set_montage(standard_montage) # attribute montage to raw object

raw.plot_sensors() # display montage

Convert EEG data to BIDS format

In [ ]:
bids_root = " " # path to new BIDS output
task = " " # task
session = " " # session
subjectID = " " # subject ID number, e.g. '028'


for ids in subjectID: # create BIDS compliant data based on subject ID number
    bids_path = BIDSPath(subject=ids, task=task, root=bids_root)
    write_raw_bids(raw, bids_path, overwrite=True, allow_preload= True, format = 'BrainVision')

References

In [ ]:
readme = op.join(bids_root, 'README')
with open(readme, 'r', encoding='utf-8-sig') as fid:
    text = fid.read()
print(text)